In [41]:
import pandas as pd
import numpy as np

In [42]:
sales = pd.read_json("../../../../DataspellProjects/reviews_Electronics_5.json", lines=True)

In [43]:
sales = sales[['reviewerID', 'asin', 'overall']]

In [44]:
asin_sales = sales.groupby(by=['asin'], dropna=False).count()
asin_sales = asin_sales[asin_sales.apply(lambda x: x > 500)].dropna().drop('overall', axis=1).rename(columns = {'reviewerID': 'review_count'})

In [45]:
sales = sales.assign(over500=sales.asin.isin(asin_sales.index).astype(str))

In [46]:
sales = sales[sales['over500'] == "True"].drop('over500', axis = 1)

In [47]:
from sklearn.model_selection import train_test_split
train_sales, test_sales = train_test_split(sales, test_size=0.25)

In [48]:
train_sales_pivot = train_sales.pivot_table(index='reviewerID', columns='asin', values='overall', fill_value=0)

In [49]:
X = train_sales_pivot.T
X_index = X.index

In [50]:
train_sales_pivot.shape

(95498, 226)

In [51]:
corr = pd.read_csv("../../../../DataspellProjects/FYP/RevisedCorr.csv", index_col=0)

In [52]:
corr.head()

,B00004T8R2,B00004ZCJE,B00005T3G0,B00006RVPW,B00007E7JU,B00007EDZG,B00007M1TZ,B00009R6TA,B0000BZL1P,B00017LSPI,...,B00BGGDVOO,B00BQ4F9ZA,B00BT7RAPG,B00BWF5U0M,B00D5Q75RC,B00DR0PDNE,B00DVFLJDS,B00E055H5O,B00E3W15P0,B00GTGETFG
asin,,,,,,,,,,,,,,,,,,,,,
B00004T8R2,1.000000,-0.037205,0.273364,0.316396,-0.049178,0.125412,0.594257,0.077238,0.262085,0.233406,...,-0.032316,0.055327,0.433559,0.426263,0.199094,0.140641,0.184589,0.135814,0.150660,0.220754
B00004ZCJE,-0.037205,1.000000,-0.174312,-0.028199,0.095305,0.808419,0.073975,0.499657,0.369860,0.565390,...,-0.009444,0.083899,-0.061911,-0.026930,-0.059446,0.016587,-0.170191,0.050777,0.009165,-0.086188
B00005T3G0,0.273364,-0.174312,1.000000,0.231551,-0.023427,0.118387,0.134753,-0.020159,0.226419,0.223711,...,0.013847,0.146434,0.501032,0.338589,0.314345,0.074207,0.090018,0.326122,0.045167,0.087036
B00006RVPW,0.316396,-0.028199,0.231551,1.000000,-0.030959,0.211244,0.489592,0.109282,0.284682,0.158018,...,0.175352,0.476912,0.212479,0.431233,0.483357,0.182336,0.075778,0.282929,0.290836,0.468848
B00007E7JU,-0.049178,0.095305,-0.023427,-0.030959,1.000000,-0.024138,-0.055566,0.805229,0.291818,0.396591,...,-0.029004,0.040531,0.023080,-0.084880,0.135781,0.014933,0.120073,0.055848,0.058263,0.045545


In [53]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = pairwise_distances(X, metric='cosine')

In [54]:
stacked_sales = train_sales_pivot.stack()
non_zero = stacked_sales[stacked_sales.ne(0)]

non_zero

reviewerID             asin      
A00101847G3FJTWYGNQA   B009NB8WR0    5
A00166281YWM98A3SVD55  B0074BW614    5
A0046696382DWIPVIWO0K  B00E055H5O    5
A00473363TJ8YSZ3YAGG9  B003NR57BY    4
A005721627VX5W2COKKK2  B005QX7KYU    5
                                    ..
AZZYW4YOE1B6E          B000CKVOOY    5
                       B003ELYQGG    5
                       B004GF8TIK    4
                       B004HHICKC    4
                       B009USAJCC    5
Length: 153831, dtype: int64

In [55]:
grouped = non_zero.groupby("reviewerID")

# Filter the groups based on their size
filtered_groups = grouped.filter(lambda x: x.size > 3)

In [56]:
# actual_ratings= filtered_groups.reset_index().set_index('reviewerID').rename(columns={0: "rating"})


In [57]:
unparsed_sales = filtered_groups.to_dict()
unparsed_sales

{('A0110255Y4ER0RTKEKXS', 'B001NJ0D0Y'): 5,
 ('A0110255Y4ER0RTKEKXS', 'B0034CL2ZI'): 5,
 ('A0110255Y4ER0RTKEKXS', 'B0058UUR6E'): 5,
 ('A0110255Y4ER0RTKEKXS', 'B008NMCPTQ'): 5,
 ('A100UD67AHFODS', 'B000LRMS66'): 5,
 ('A100UD67AHFODS', 'B000Q8UAWY'): 5,
 ('A100UD67AHFODS', 'B002Y6CJJ8'): 4,
 ('A100UD67AHFODS', 'B00483WRZ6'): 5,
 ('A100UD67AHFODS', 'B005CLPP8E'): 5,
 ('A100UD67AHFODS', 'B005DOK8NW'): 5,
 ('A100UD67AHFODS', 'B0097BEFYA'): 5,
 ('A100UD67AHFODS', 'B009LL9VDG'): 5,
 ('A100WO06OQR8BQ', 'B00006RVPW'): 5,
 ('A100WO06OQR8BQ', 'B000HPV3RW'): 5,
 ('A100WO06OQR8BQ', 'B000VX6XL6'): 5,
 ('A100WO06OQR8BQ', 'B002SZEOLG'): 5,
 ('A100WO06OQR8BQ', 'B003ES5ZUU'): 5,
 ('A100WO06OQR8BQ', 'B005CLPP84'): 1,
 ('A100WO06OQR8BQ', 'B0088CJT4U'): 4,
 ('A100WO06OQR8BQ', 'B00AJHDZSI'): 5,
 ('A100WO06OQR8BQ', 'B00BGGDVOO'): 4,
 ('A1018D502E27E6', 'B002UT42UI'): 2,
 ('A1018D502E27E6', 'B005CLPP84'): 2,
 ('A1018D502E27E6', 'B009OBCAW2'): 4,
 ('A1018D502E27E6', 'B00BGGDVOO'): 3,
 ('A102SVOH0JD2RP', 'B0010

In [58]:
actual_rating = {}
for (reviewerID, asin), rating in unparsed_sales.items():
    if reviewerID not in actual_rating:
        actual_rating[reviewerID] = {}
    actual_rating[reviewerID][asin] = rating


In [59]:
actual_rating

{'A0110255Y4ER0RTKEKXS': {'B001NJ0D0Y': 5,
  'B0034CL2ZI': 5,
  'B0058UUR6E': 5,
  'B008NMCPTQ': 5},
 'A100UD67AHFODS': {'B000LRMS66': 5,
  'B000Q8UAWY': 5,
  'B002Y6CJJ8': 4,
  'B00483WRZ6': 5,
  'B005CLPP8E': 5,
  'B005DOK8NW': 5,
  'B0097BEFYA': 5,
  'B009LL9VDG': 5},
 'A100WO06OQR8BQ': {'B00006RVPW': 5,
  'B000HPV3RW': 5,
  'B000VX6XL6': 5,
  'B002SZEOLG': 5,
  'B003ES5ZUU': 5,
  'B005CLPP84': 1,
  'B0088CJT4U': 4,
  'B00AJHDZSI': 5,
  'B00BGGDVOO': 4},
 'A1018D502E27E6': {'B002UT42UI': 2,
  'B005CLPP84': 2,
  'B009OBCAW2': 4,
  'B00BGGDVOO': 3},
 'A102SVOH0JD2RP': {'B00109Y2DQ': 5,
  'B002JQNXZC': 4,
  'B008OHNZI0': 4,
  'B009SYZ8OC': 5},
 'A1053FVPAZUKMF': {'B000UEZ36W': 4,
  'B008NMCPTQ': 3,
  'B009USAJCC': 5,
  'B00GTGETFG': 5},
 'A107K10JTO8NSH': {'B0002L5R78': 5,
  'B0015DYMVO': 5,
  'B001F7AJKI': 4,
  'B004OB0EBQ': 3},
 'A10AAULK3SQEK1': {'B000S5Q9CA': 5,
  'B005CPGHAA': 5,
  'B005DIBHA6': 5,
  'B007IV7KRU': 5},
 'A10AGLJYHNUKZA': {'B002V8C3W2': 5,
  'B003M0NURK': 5,
  'B004

In [60]:
corr_matrix = corr.to_dict()

In [61]:
def get_avg(reviewerID):
    sum = 0
    for item in actual_rating[reviewerID]:
        sum += actual_rating[reviewerID][item]
        
    return(sum / len(actual_rating[reviewerID]))

In [62]:
def rmse(predictions, ground_truth):
    predictions = np.array(predictions)
    ground_truth = np.array(ground_truth)
    return np.sqrt(np.mean((predictions - ground_truth) ** 2))

In [63]:
def lenient():
    predictions = []
    ground_truths = []
    
    for reviewer in actual_rating:
        for item in actual_rating[reviewerID]:
            
            ground_truth = actual_rating[reviewerID][item]

            # Initialize the predicted rating to the average rating for the user
            user_average = get_avg(reviewerID)
            prediction = user_average
        
            for corr_item in actual_rating[reviewer]:
            # If the item has a non-zero correlation with item i
                if corr_item !=item  and corr_matrix[corr_item][item] > 0:
                # Add the product of the correlation and the difference between the rating and the average rating to the prediction
                    prediction += corr_matrix[corr_item][item] * (actual_rating[reviewerID][item] - user_average)

            predictions.append(prediction)
            ground_truths.append(ground_truth)
        
    return rmse(predictions, ground_truths)

In [102]:
def predict_rating(correlations, ratings):
  predicted_rating = 0
  total_correlation = 0
  for i in range(len(correlations)):
    if correlations[i] > 0:
      predicted_rating += correlations[i] * ratings[i]
      total_correlation += correlations[i]
    
  # print(predicted_rating, total_correlation) 
  # print(*correlations, *ratings)
  # print("Predicted: " + str(predicted_rating / total_correlation))
  if predicted_rating > 0:
    return predicted_rating / total_correlation
  else: 
    return 0

In [104]:
def harsh():
    predictions = []
    ground_truths = []
    for reviewer in actual_rating:
        
        # This creates a list of the ratings that the current reviewer has given to all items. 
        # This will be used later as the ground truth for evaluating the accuracy of the predictions.
        truth_array = [actual_rating[reviewer][item] for item in actual_rating[reviewer]]
        
        # This creates a list of the items that the current reviewer has rated.
        items = [item for item in actual_rating[reviewer]]
        
        # This creates a list of lists, where each inner list represents the items that the current reviewer has rated, 
        # excluding the item at the corresponding index in the items list. 
        # For example, if the items list is [a, b, c], 
        # then corr_items will be [[b, c], [a, c], [a, b]].
        corr_items = [items[:i] + items[i+1:] for i in range(len(items))]
        
        # This creates a list of lists, 
        # where each inner list represents the similarity between the item at the corresponding index in the items list and all other items that the current reviewer has rated. 
        # For example, if the items list is [a, b, c] and corr_matrix[a][b] is 0.5, corr_matrix[a][c] is 0.7, corr_matrix[b][c] is 0.3, 
        # then pred_corr will be [[0.5, 0.7], [0.7, 0.3], [0.5, 0.3]].
        pred_corr = [[corr_matrix[items[i]][corr_item] for corr_item in corr_items[i]] for i in range(len(items))]
        
        # This creates a list of predictions for the ratings that the current reviewer will give to all items. 
        # It does this by calling the predict_rating function on each pair of similarity values and 
        # corresponding ratings from pred_corr and the list comprehension [actual_rating[reviewer][corr_item] for corr_item in corr_items[i]].
        prediction = [predict_rating(pred_corr[i], [actual_rating[reviewer][corr_item] for corr_item in corr_items[i]]) for i in range(len(items))]
        
        
        predictions.extend(prediction)
        ground_truths.extend(truth_array)
        
    return rmse(predictions, ground_truths)

In [105]:
print("Recommendation System Accuracy Through RMSE Lenient: " + str(lenient()))
print("Recommendation System Accuracy Through RMSE Harsh: " + str(harsh()))

Recommendation System Accuracy Through RMSE Lenient: 0.3471288284211668
Recommendation System Accuracy Through RMSE Harsh: 1.0799780023949475
